# Data Preparation
The following need to be done:
* Load the relevant data sets from file
* Join them into a single data set
* Add additional computed features to the data
* Write the prepared data to file

### Set up the environment
We need a certain set of common libraries for the tasks to be performed. These are imported below. If an import statement errors, you will need to install the library in your environment using the command line command `pip install <library>`.

In [1]:
print('Setting up environment and variables.', flush=True)
import pandas as pd
import os
import numpy as np
import datetime
import time
import yaml
from sqlalchemy import create_engine

df_sensor_data = None

Setting up environment and variables.


### Set up the variables
Change the values of the variables below to suit the files (names and directory location) to be loaded.

In [2]:
## Currently in unix format as docker containers run on debian
config_file = os.path.normpath('../config.yml')
temperature_file = os.path.normpath('../0_data/TempData_2_10_2016.txt')
humidity_file = os.path.normpath('../0_data/HumidData_2_10_2016.txt')
joined_data_file = os.path.normpath('../0_data/TempHumdCombined.csv')
nest_static_file = os.path.normpath('../0_data/NestCharacteristic-Static.csv')
nest_seasonal_file = os.path.normpath('../0_data/NestCharacteristic-Seasonal.csv')
breeding_data_file = os.path.normpath('../0_data/BreedingDataCombined.csv')

In [3]:
# initialise the config.yml file
# try:
#     with open(config_file, 'r') as ymlfile:
#         config = yaml.load(ymlfile)
# except IOError:
#     print('Config file can\'t be found.')

In [4]:
# set the database connection parameters based on the config.ini file
# if ymlfile is not None:
#     host = config['PostgreSQL']['host']
#     port = config['PostgreSQL']['port']
#     dbname = config['PostgreSQL']['dbname']
#     user  = config['PostgreSQL']['user']
#     password = config['PostgreSQL']['password']
    
#     # establish connection to the postgres database using the generated connection string
#     engine = create_engine(r"postgresql://"+user+":"+password+"@"+host+"/"+dbname)

## Read in the NestCharacteristic-Static data
This is the real nest master data to which everything else is joined.

In [5]:
print(str(time.ctime()), 'Loading the Nest Characteristic (Static) data.', flush=True)

file_size = os.path.getsize(nest_static_file)
print('\n{0} Nest Characteristic (Static) file is {1:.1f} MB.'.format(str(time.ctime()), (file_size/1000000)))

if file_size > 5000000: # over 5mb
    print(str(time.ctime()), 'Loading into memory. Please be patient. ', flush=True)
else:
    print(str(time.ctime()), 'Loading into memory. ', flush=True)
data_types = {'nest_id': str,
              'nest_type': str,
              'distance_to_boardwalk_m': np.int32,
              'distance_to_vegetation_m': np.int32,
              'distance_to_landfall': np.int32,
              'entrance_bearing': np.int32,
              'box_height_mm': np.int32,
              'box_length_mm': np.int32,
              'box_width_mm': np.int32,
              'box_wall_width_mm': np.int32,
              'box_lid_depth': np.int32,
              'internal_height_mm': np.int32,
              'internal_width_mm': np.int32,
              'internal_length_mm': np.int32,
              'entrance_height': np.int32,
              'entrance_width': np.int32,
              'entrance_length': np.int32,
              'vents': np.int32,
              'box_vol_L': np.int32,
              'box_area_cm2': np.int32,
              'box_has_tunnel': np.int32,
              'shape': str,
              'elevation': np.float32,
              'easting': np.float32,
              'northing': np.float32,
              'aspect': np.float32,
              'slope': np.float32,
              'duration_of_insolation': np.float32,
              'comment': str}
df_nest_static = pd.read_csv(nest_static_file, 
                             header=0, 
                             encoding='utf-8',
                             error_bad_lines=False,
                             warn_bad_lines=True)

if df_nest_static is not None:
    print(str(time.ctime()), 'Success: loaded {0:,} records.'.format(len(df_nest_static)))
else:
    print(str(time.ctime()), '### FAILED! ###')

Sun Nov  6 21:04:23 2016 Loading the Nest Characteristic (Static) data.

Sun Nov  6 21:04:23 2016 Nest Characteristic (Static) file is 0.0 MB.
Sun Nov  6 21:04:23 2016 Loading into memory. 
Sun Nov  6 21:04:23 2016 Success: loaded 241 records.


### Update and cleanse some fields
* Make sure all the nest IDs are uppercase
* Create `box_vol_L`
* create `box_area_cm2`

In [6]:
# make sure the nest IDs are all caps
df_nest_static['nest_id'] = df_nest_static['nest_id'].apply(lambda x: x.upper())
# calc the volume
df_nest_static['box_vol_L'] = df_nest_static['internal_width_mm'] * df_nest_static['internal_height_mm'] * df_nest_static['internal_length_mm'] / 1000000
# calc the floor area
df_nest_static['box_area_cm2'] = df_nest_static['internal_width_mm'] * df_nest_static['internal_length_mm'] / 100

## Read in the NestCharacteristic-Seasonal data

In [7]:
print(str(time.ctime()), 'Loading the Nest Characteristic (Seasonal) data.', flush=True)

file_size = os.path.getsize(nest_seasonal_file)
print('\n{0} Nest Characteristic (Seasonal) file is {1:.1f} MB.'.format(str(time.ctime()), (file_size/1000000)))

if file_size > 5000000: # over 5mb
    print(str(time.ctime()), 'Loading into memory. Please be patient. ', flush=True)
else:
    print(str(time.ctime()), 'Loading into memory. ', flush=True)

data_types = {'type': str,
              'nest_id': str,
              'BoxSeasYear': str,
              'date': str,
              'year': str,
              'season': str,
              'BoxCoverTotal': np.int32,
              'BoxCoverDead': np.int32,
              'BoxWood': np.int32,
              'BoxWoodDead': np.int32,
              'BoxVeg': np.int32,
              'BoxVegDead': np.int32,
              'QuadCoverTotal': np.int32,
              'QuadCoverDead': np.int32,
              'QuadWood': np.int32,
              'QuadWoodDead': np.int32,
              'QuadVeg': np.int32,
              'QuadVegDead': np.int32,
              'comments': str
             }
df_nest_seasonal = pd.read_csv(nest_seasonal_file, 
                             header=0, 
                             encoding='utf-8',
                             parse_dates=['date'],
                             error_bad_lines=False,
                             warn_bad_lines=True)

if df_nest_static is not None:
    print(str(time.ctime()), 'Success: loaded {0:,} records.'.format(len(df_nest_seasonal)))
else:
    print(str(time.ctime()), '### FAILED! ###')

Sun Nov  6 21:04:23 2016 Loading the Nest Characteristic (Seasonal) data.

Sun Nov  6 21:04:23 2016 Nest Characteristic (Seasonal) file is 0.1 MB.
Sun Nov  6 21:04:23 2016 Loading into memory. 
Sun Nov  6 21:04:24 2016 Success: loaded 1,711 records.


### Update and cleanse fields
* Nest IDs to be all uppercase
* recalculate the `year` and `season`
* create the unique ID `BoxSeasYear`

In [8]:
# make sure the nest IDs are all caps
df_nest_seasonal['nest_id'] = df_nest_seasonal['nest_id'].apply(lambda x: x.upper())

# recalculate year (because was manually created)
df_nest_seasonal['year'] = df_nest_seasonal['date'].apply(lambda x: x.year)

# recalculate season
def season(date):
    if date.month >= 3 and date.month <= 5:
        return 'AUTUMN'
    elif date.month >= 6 and date.month <= 8:
        return 'WINTER'
    elif date.month >= 9 and date.month <= 11:
        return 'SPRING'
    elif (date.month >= 1 and date.month <= 2) or date.month == 12:
        return 'SUMMER'
    else:
        return None
    
df_nest_seasonal['season'] = df_nest_seasonal['date'].apply(lambda x: season(x))

# calc the unique ID
df_nest_seasonal['BoxSeasYear'] = df_nest_seasonal['nest_id'] + df_nest_seasonal['season'] + df_nest_seasonal['year'].apply(lambda x: str(x))

## Read in the BreedingDataCombined

In [9]:
print(str(time.ctime()), 'Loading the Breeding data.', flush=True)

file_size = os.path.getsize(breeding_data_file)
print('\n{0} Breeding file is {1:.1f} MB.'.format(str(time.ctime()), (file_size/1000000)))

if file_size > 5000000: # over 5mb
    print(str(time.ctime()), 'Loading into memory. Please be patient. ', flush=True)
else:
    print(str(time.ctime()), 'Loading into memory. ', flush=True)

data_types = {'nest_id': str,
              'observation_date': str,
              'ActivityStatus': np.int32,
              'adult': np.int32,
              'clutch': np.int32,
              'eggs': np.int32,
              'ChicksAlive': np.int32,
              'ChicksDead': np.int32,
              'ChicksAge': np.int32,
              'ChicksFledge': np.int32,
              'ChicksMissing': np.int32,
              'ContentsNotVisible': np.int32,
              'EggLayDate': str,
              'IDChick1': np.int32,
              'MassChick1': np.int32,
              'IDChick2': np.int32,
              'MassChick2': np.int32,
              'comments': str
             }
df_breeding = pd.read_csv(breeding_data_file, 
                             header=0, 
                             encoding='utf-8',
                             parse_dates=['observation_date', 'EggLayDate'],
                             error_bad_lines=False,
                             warn_bad_lines=True)

if df_breeding is not None:
    print(str(time.ctime()), 'Success: loaded {0:,} records.'.format(len(df_breeding)))
else:
    print(str(time.ctime()), '### FAILED! ###')

Sun Nov  6 21:04:24 2016 Loading the Breeding data.

Sun Nov  6 21:04:24 2016 Breeding file is 0.1 MB.
Sun Nov  6 21:04:24 2016 Loading into memory. 
Sun Nov  6 21:04:24 2016 Success: loaded 3,575 records.


### Clean and add columns
* `year` is year of `observation_date`
* make `nest_id` uppercase

In [10]:
# make sure the nest IDs are all caps
df_breeding['nest_id'] = df_breeding['nest_id'].apply(lambda x: x.upper())

# create year field
df_breeding['year'] = df_breeding['observation_date'].apply(lambda x: x.year)

### Aggregate the Breeding data to get annual stats
* _nest_id_
* _year_
* _clutch_
* clutch_count
* egg_count
* chick_count
* fletch_count
* lay_date
* age_at_fletching
* mass_at_fletching_chick1
* mass_at_fletching_chick2
* chick_id1
* chick_id2

In [11]:
print(str(time.ctime()), 'Aggregating breeding data to get annual stats.', flush=True)

# get the clutches per nest and year
# [[chosen columns]] -> groupby -> apply max -> add suffix -> remove multi-index
df_clutch_count = df_breeding[['nest_id', 'year', 'clutch']
                             ].groupby(['nest_id', 'year']).max().add_suffix('_count').reset_index()

# get the annual stats per nest, year and clutch
temp = df_breeding[['nest_id', 'year', 'clutch', 'eggs', 'ChicksAlive', 'ChicksFledge', 'EggLayDate', 'ChicksAge', 'MassChick1', 'MassChick2', 'IDChick1', 'IDChick2']].copy()
df_breeding_gb = temp.groupby(['nest_id', 'year', 'clutch']).max().reset_index()
df_breeding_gb.rename(columns = {'eggs': 'egg_count', 'ChicksAlive': 'chick_count', 'ChicksFledge': 'fledge_count', 
                     'EggLayDate': 'lay_date', 'ChicksAge': 'age_at_fledging', 'MassChick1': 'mass_at_fletching_chick1', 
                     'MassChick2': 'mass_at_fletching_chick1'}
          , inplace=True)



Sun Nov  6 21:04:24 2016 Aggregating breeding data to get annual stats.


## Load the temperature data
Read the temperature data file into memory and report on success/failure.

In [12]:
if os.path.isfile(joined_data_file):
    print(str(time.ctime()), 'Combined temp and humidity file found. Skipping the temp data load.', flush=True)
else:
    column_names = ['recnum', 'datetime', 'temp_c', 'nest_id']
    data_types = {'recnum': np.int32, 
                  'datetime': str, 
                  'temp_c': np.float32, 
                  'nest_id': str}
    file_size = os.path.getsize(temperature_file)
    print('\n{0} Temperature is {1:.1f} MB.'.format(str(time.ctime()), 
                                                                                     (file_size/1000000)))

    if file_size > 5000000: # over 5mb
        print(str(time.ctime()), 'Loading into memory. Please be patient. ', flush=True)
    else:
        print(str(time.ctime()), 'Loading into memory. ', flush=True)

    df_temp = pd.read_csv(temperature_file,
                         names=column_names,
                         usecols=[0,1,2,3],
                         dtype=data_types,
#                          nrows=10000,               # for testing only
                          parse_dates=['datetime'],
                          dayfirst=True,
                          encoding='utf-8',
                          error_bad_lines=False,
                          warn_bad_lines=True
                         )

    if df_temp is not None:
        print(str(time.ctime()), 'Success: loaded {0:,} records.'.format(len(df_temp)))
    else:
        print(str(time.ctime()), '### FAILED! ###')
    
    # make sure the nest IDs are all caps
    df_temp['nest_id'] = df_temp['nest_id'].apply(lambda x: x.upper())

Sun Nov  6 21:04:24 2016 Combined temp and humidity file found. Skipping the temp data load.


In [13]:
if os.path.isfile(joined_data_file):
    print(str(time.ctime()), 'Combined temp and humidity file found. Skipping the humidity data load.', flush=True)
else:
    column_names = ['recnum', 'datetime', 'humidity', 'nest_id']
    data_types = {'recnum': np.int32, 
                  'datetime': str, 
                  'humidity': np.float32, 
                  'nest_id': str}
    file_size = os.path.getsize(humidity_file)
    print('\n{0} Humidity file is {1:.1f} MB.'.format(str(time.ctime()),
                                                      (file_size/1000000)), flush=True)

    if file_size > 5000000: # over 5mb
        print(str(time.ctime()), 'Loading into memory. Please be patient. ', flush=True)
    else:
        print(str(time.ctime()), 'Loading into memory. ', flush=True, end='')

    df_humd = pd.read_csv(humidity_file,
                         names=column_names,
                         usecols=[0,1,2,3],
                         dtype=data_types,
#                          nrows=10000,               # for testing only
                          parse_dates=['datetime'],
                          dayfirst=True,
                          encoding='utf-8',
                          error_bad_lines=False,
                          warn_bad_lines=True
                         )

    if df_humd is not None:
        print(str(time.ctime()), 'Success: loaded {0:,} records.'.format(len(df_humd)))
    else:
        print(str(time.ctime()), '### FAILED! ###')
    
    # make sure the nest IDs are all caps
    df_humd['nest_id'] = df_humd['nest_id'].apply(lambda x: x.upper())

Sun Nov  6 21:04:24 2016 Combined temp and humidity file found. Skipping the humidity data load.


In [14]:
if os.path.isfile(joined_data_file):
    print(str(time.ctime()), 'Combined temp and humidity file found. Skipping the temp and humidity data join.', flush=True)
else:
    print('\n{0} Joining the temperature and humidity data sets.'.format(str(time.ctime())), flush=True)
    df_sensor_data = pd.merge(left=df_temp,
                            right=df_humd,
                            how='outer',
                            on=['nest_id', 'datetime'], # both have same keys
                            left_on=None, # same key names: don't need to specify R and L
                            right_on=None, # same key names: don't need to specify R and L
                            left_index=False, # dont' use left df index as key
                            right_index=False, # dont' use right df index as key
                            sort=True, # for efficiency do/not sort the df first
                            suffixes=['_temp', '_humd']
                            )[['nest_id', 'datetime', 'temp_c', 'humidity']] # take only these cols

    print('{0} Join complete. Here are the stats:'.format(str(time.ctime())))
    print('Records in temperature data: {0:>20,}'.format(len(df_temp)))
    print('Records in humidity data:    {0:>20,}'.format(len(df_humd)))
    print('                              -------------------')
    print('Records in joined data:      {0:>20,}'.format(len(df_sensor_data)))
    print('\nOverview:')
    gb = df_sensor_data.groupby(['nest_id'])
    print('Number of nest_ids:          {0:>20,}'.format(len(gb)))

Sun Nov  6 21:04:24 2016 Combined temp and humidity file found. Skipping the temp and humidity data join.


## Load the joined temp and humidity csv

In [15]:
if df_sensor_data is None and os.path.isfile(joined_data_file):
    data_types = {'nest_id': str,
    #               'recnum': np.int32, 
                  'datetime': str, 
                  'temp_c': np.float32,
                  'humidity': np.float32, 
                  'breeding_year': np.int32}
    file_size = os.path.getsize(joined_data_file)
    print('\n{0} Combined temp and humidity file is {1:.1f} MB.'.format(str(time.ctime()), (file_size/1000000)))

    if file_size > 5000000: # over 5mb
        print(str(time.ctime()), 'Loading into memory. Please be patient. ', flush=True)
    else:
        print(str(time.ctime()), 'Loading into memory. ', flush=True, end='')

    df_sensor_data = pd.read_csv(joined_data_file,
    #                      names=column_names,
    #                      usecols=[0,1,2,3],
                         dtype=data_types,
    #                      nrows=2048,               # for testing only
                          parse_dates=['datetime'],
                          dayfirst=True,
                          encoding='utf-8',
                          error_bad_lines=False,
                          warn_bad_lines=True
                         )

    if df_sensor_data is not None:
        print(str(time.ctime()), 'Success: loaded {0:,} records.'.format(len(df_sensor_data)))
    else:
        print(str(time.ctime()), '### FAILED! ###')


Sun Nov  6 21:04:24 2016 Combined temp and humidity file is 167.9 MB.
Sun Nov  6 21:04:24 2016 Loading into memory. Please be patient. 
Sun Nov  6 21:04:29 2016 Success: loaded 2,173,738 records.


## Calculations per-sensor reading
The following calculations are added per sensor reading:
* The `breeding_year`: same as the calendar year
* `temp_bucket` is a category for each 5C temperature range: <0, 0-5, .., 60+
* `humidity_bucket`: is a category for roughly 20% humidity ranges, based on human comfort zones
* `average_activity_phase`: the average activity conducted at the time of the observation

### To be added:
* `actual_activity_phase`: Is the current phase of breeding based on per-nest observations. 

In [16]:
def temp_bucket(temp_c):
    result = None
    if temp_c < 0:
        result = 'temp_<0'
    elif temp_c >= 0 and temp_c < 5:
        result = 'temp_0-5'
    elif temp_c >= 5 and temp_c < 10:
        result = 'temp_5-10'
    elif temp_c >= 10 and temp_c < 15:
        result = 'temp_10-15'
    elif temp_c >= 15 and temp_c < 20:
        result = 'temp_15-20'
    elif temp_c >= 20 and temp_c < 25:
        result = 'temp_20-25'
    elif temp_c >= 25 and temp_c < 30:
        result = 'temp_25-30'
    elif temp_c >= 30 and temp_c < 35:
        result = 'temp_30-35'
    elif temp_c >= 35 and temp_c < 40:
        result = 'temp_35-40'
    elif temp_c >= 40 and temp_c < 45:
        result = 'temp_40-45'
    elif temp_c >= 45 and temp_c < 50:
        result = 'temp_45-50'
    elif temp_c >= 50 and temp_c < 55:
        result = 'temp_50-55'
    elif temp_c >= 55 and temp_c < 60:
        result = 'temp_55-60'
    elif temp_c >= 60:
        result = 'temp_60+'
    return result

def humidity_bucket(humidity):
    result = None
    if humidity < 20: # lung & eye irritation in humans
        result = 'RH%_<20'
    elif humidity >= 20 and humidity < 30: # lung irritation in humans
        result = 'RH%_20-30'
    elif humidity >= 30 and humidity < 50: # low but not dangerous to humans
        result = 'RH%_30-50'
    elif humidity >= 50 and humidity < 60: # human ideal comfort zone 
        result = 'RH%_50-60'
    elif humidity >= 60 and humidity < 80: # humid
        result = 'RH%_60-80'
    elif humidity >= 80 and humidity < 100: # v humid
        result = 'RH%_80-100'
    elif humidity >= 100: # dripping 
        result = 'RH%_100+'
    return result

def average_activity_phase(sensor_datetime):
    '''
    Returns the current phase of breeding based on per-nest observations. Phases are generally:
    1 Jan - 31 Mar: moulting
    1 Apr - 31 May: nest building
    1 Jun - 30 Jun: laying
    1 Jul - 7 Aug: incubating
    8 Aug - 30 Sep: rearing
    1 Oct - 30 Oct: fledging
    1 Nov - 31 Dec: post-fledging
    There can be two lays per season. The second lay is not considered in the average timeframes 
    above.
    '''
    if sensor_datetime is None:
        return None
    elif sensor_datetime.month >= 1 and sensor_datetime.month <= 3:
        return 'moulting'
    elif sensor_datetime.month >= 4 and sensor_datetime.month >= 5:
        return 'nest building'
    elif sensor_datetime.month == 6:
        return 'laying'
    elif sensor_datetime.month == 7:
        return 'incubating'
    elif sensor_datetime.month == 8 and date(sensor_datetime.year, sensor_datetime.month, sensor_datetime.day) <= date(sensor_datetime.year, 8, 7):
        return 'incubating'
    elif sensor_datetime.month == 8 and date(sensor_datetime.year, sensor_datetime.month, sensor_datetime.day) > date(sensor_datetime.year, 8, 7):
        return 'rearing'
    elif sensor_datetime.month >= 9:
        return 'rearing'
    elif sensor_datetime.month >= 10:
        return 'fledging'
    elif sensor_datetime.month in [11, 12]:
        return 'post-fledging'
    else:
        return 'unknown'

In [17]:
if os.path.isfile(joined_data_file):
    print(str(time.ctime()), 'Combined temp and humidity file found. Skipping calculated fields.', flush=True)
else:
    # add the breeding_year (same as financial year): 
    print(str(time.ctime()), 'Calculating breeding year.', end='', flush=True)
    df_sensor_data['breeding_year'] = df_sensor_data['datetime'].apply(lambda x: x.year)
    print(' Done.', flush=True)

    # # add the average breeding phases 
    print(str(time.ctime()), 'Calculating average activity periods.', end='', flush=True)
    df_sensor_data['average_activity_period'] = df_sensor_data['datetime'].apply(average_activity_phase)
    print(' Done.', flush=True)

    # Add flags for various temperature ranges. 
    # These are summed to give the amount of time in the temp band
    print(str(time.ctime()), 'Calculating temperature buckets.', end='', flush=True)
    df_sensor_data['temp_bucket'] = df_sensor_data['temp_c'].apply(temp_bucket)
    print(' Done.', flush=True)

    # # Add flags for various humidity ranges. 
    # # These are summed to give the amount of time in the humidity band
    print(str(time.ctime()), 'Calculating humidity buckets.', end='', flush=True)
    df_sensor_data['humidity_bucket'] = df_sensor_data['humidity'].apply(humidity_bucket)
    print(' Done.', flush=True)

Sun Nov  6 21:04:29 2016 Combined temp and humidity file found. Skipping calculated fields.


### Write the created data to file to use again in future.

In [18]:
if os.path.isfile(joined_data_file):
    print(str(time.ctime()), 'Combined temp and humidity file found. Skipping writing the combined data to file.', flush=True)
else:
    print('\n{0} Writing the joined dataset to csv.'.format(str(time.ctime())), flush=True)
    df_sensor_data.to_csv(path_or_buf=joined_data_file,
                     sep=',',
                     na_rep='',
                     float_format='%.3f',
                     header=True,
                     index=False,
                     mode='w',
                     encoding='utf-8')
    print('{0} File written: {1} ({2:.1f}MB)'.format(str(time.ctime()), str(joined_data_file), os.path.getsize(joined_data_file)/1000000), flush=True)

Sun Nov  6 21:04:29 2016 Combined temp and humidity file found. Skipping writing the combined data to file.


# Join the data sets

## Join the NestCharacteristic Static and Seasonal data

In [19]:
print('{0} Joining the Nest Characteristic (Seasonal and Static) data sets .'.format(str(time.ctime())), flush=True)
df_nest_joined = pd.merge(left=df_nest_seasonal,
                            right=df_nest_static,
                            how='left',
                            on=['nest_id'], # both have same keys
                            left_on=None, # same key names: don't need to specify R and L
                            right_on=None, # same key names: don't need to specify R and L
                            left_index=False, # dont' use left df index as key
                            right_index=False, # dont' use right df index as key
                            sort=True, # for efficiency do/not sort the df first
                            suffixes=['_seasonal', '_static']
                            )
if df_nest_joined is not None:
    print('{0} Join complete. Here are the stats:'.format(str(time.ctime())))
    print('Records in seasonal data:    {0:>20,}'.format(len(df_nest_seasonal)))
    print('Records in static data:      {0:>20,}'.format(len(df_nest_static)))
    print('                              -------------------')
    print('Records in joined data:      {0:>20,}'.format(len(df_nest_joined)))
    print('\nOverview:')
    gb = df_nest_joined.groupby(['nest_id'])
    print('Number of nest_ids:          {0:>20,}'.format(len(gb)))
else:
    print('{0} JOIN FAILED!!!.'.format(str(time.ctime())), flush=True)

Sun Nov  6 21:04:29 2016 Joining the Nest Characteristic (Seasonal and Static) data sets .
Sun Nov  6 21:04:29 2016 Join complete. Here are the stats:
Records in seasonal data:                   1,711
Records in static data:                       241
                              -------------------
Records in joined data:                     1,711

Overview:
Number of nest_ids:                           193


## Join the breeding stats together
Clutch counts per year and annual clutch survival stats

In [20]:
# join the clutch count on to the annual stats
print('\n{0} Merging the aggregated breeding stats.'.format(str(time.ctime())), flush=True)
df_breeding_annual_stats = pd.merge(left=df_breeding_gb,
                        right=df_clutch_count,
                        how='left',
                        on=['nest_id', 'year'], # both have same keys
#                             left_on=None, # same key names: don't need to specify R and L
#                             right_on=None, # same key names: don't need to specify R and L
#                             left_index=False, # dont' use left df index as key
#                             right_index=False, # dont' use right df index as key
                        sort=True # for efficiency do/not sort the df first
#                             suffixes=['_temp', '_humd']
                        )#[['nest_id', 'datetime', 'temp_c', 'humidity']] # take only these cols

print('{0} Join complete. Here are the stats:'.format(str(time.ctime())))
print('Records in annual stats data: {0:>20,}'.format(len(df_breeding_gb)))
print('Records in clutch count data: {0:>20,}'.format(len(df_clutch_count)))
print('                              -------------------')
print('Records in joined data:       {0:>20,}'.format(len(df_breeding_annual_stats)))
print('\nOverview:')
gb = df_breeding_annual_stats.groupby(['nest_id'])
print('Number of nest_ids in clutch count:   {0:>12,}'.format(len(df_clutch_count.groupby(['nest_id']))))
print('Number of nest_ids in breeding stats: {0:>12,}'.format(len(df_breeding_gb.groupby(['nest_id']))))
print('Number of nest_ids in joined:         {0:>12,}'.format(len(df_breeding_annual_stats.groupby(['nest_id']))))


Sun Nov  6 21:04:29 2016 Merging the aggregated breeding stats.
Sun Nov  6 21:04:29 2016 Join complete. Here are the stats:
Records in annual stats data:                  267
Records in clutch count data:                  302
                              -------------------
Records in joined data:                        267

Overview:
Number of nest_ids in clutch count:            129
Number of nest_ids in breeding stats:          121
Number of nest_ids in joined:                  121


## Join the Nest data (seasonal and static) to the Breeding stats

In [21]:
# join the annual clutch and breeding stats onto the full seasonal and static nest data
print('\n{0} Merging the aggregated breeding stats to the static and seasonal nest data.'.format(str(time.ctime())), flush=True)
df_nest_and_breeding = pd.merge(left=df_nest_joined,
                        right=df_breeding_annual_stats,
                        how='left',
                        on=['nest_id'], # both have same keys
#                             left_on=None, # same key names: don't need to specify R and L
#                             right_on=None, # same key names: don't need to specify R and L
#                             left_index=False, # dont' use left df index as key
#                             right_index=False, # dont' use right df index as key
                        sort=True # for efficiency do/not sort the df first
#                             suffixes=['_temp', '_humd']
                        )#[['nest_id', 'datetime', 'temp_c', 'humidity']] # take only these cols

print('{0} Join complete. Here are the stats:'.format(str(time.ctime())))
print('Records in nest data:                 {0:>12,}'.format(len(df_nest_joined)))
print('Records in breeding stats data:       {0:>12,}'.format(len(df_breeding_annual_stats)))
print('                                      ------------')
print('Records in joined data:               {0:>12,}'.format(len(df_nest_and_breeding)))
print('\nOverview:')
gb = df_breeding_annual_stats.groupby(['nest_id'])
print('Number of nest_ids in nest data:      {0:>12,}'.format(len(df_nest_joined.groupby(['nest_id']))))
print('Number of nest_ids in breeding stats: {0:>12,}'.format(len(df_breeding_annual_stats.groupby(['nest_id']))))
print('Number of nest_ids in joined:         {0:>12,}'.format(len(df_nest_and_breeding.groupby(['nest_id']))))


Sun Nov  6 21:04:29 2016 Merging the aggregated breeding stats to the static and seasonal nest data.
Sun Nov  6 21:04:29 2016 Join complete. Here are the stats:
Records in nest data:                        1,711
Records in breeding stats data:                267
                                      ------------
Records in joined data:                      2,927

Overview:
Number of nest_ids in nest data:               193
Number of nest_ids in breeding stats:          121
Number of nest_ids in joined:                  193


## Aggregate the sensor data into stats per breeding phase
To understand the effect of nest conditions (from sensor data) in the choice of nest and breeding success of the nest, we need to break up the stats into:
* *annual stats* which represent the averages, spikes etc for the entire year. These give an understanding of the nest itself.
* *phase stats* which represent the conditions during specific phases of the breeding cycle. E.g. during nesting, during incubation, during rearing. To get these phase stats, we need to get the phase boundary dates from the breeding observation data.

The nest sensor readings are aggregated to summarise the nest conditions by `nest`, `breeding_year` and `activity_phase`.

### Get the phase dates
Summarise the breeding data to obtain the following:
* list of all nests (regardless of breeding activity)
* the `nesting_date` for each nest in each year
* the `egg_lay_date` for each nest, year and clutch
* the `hatch_date` for each nest, year and clutch
* the `fledge_date` for each nest, year and clutch

Join these all back together to get the phase dates all in one place, then join the combined result on to the sensor data table and calculate the phase in which each sensor reading occurred.
This will take a while.

**Issue: Nesting dates dont work: the second clutch will have first nesting date and the first obs for many nests is after the lay date, so nesting_date > lay_date**

In [22]:
# for each nest, year and clutch, get the following:
# first activity_status date (nesting_date), EggLayDate, hatch_date, fledge_date
# nesting_date, hatch_date, fledge_date are the min observation_date per nest, year, clutch where the value is not NaN

print('{0} Calculating the breeding phase dates for each nest and year.'.format(str(time.ctime())), flush=True)
# all observed nests
df_all_nests = df_nest_static[['nest_id']].drop_duplicates()

# egg_lay_date
gb_lay_date = df_breeding[['nest_id', 'year', 'clutch', 'EggLayDate']
                         ].groupby(['nest_id', 'year', 'clutch']).min().reset_index()
gb_lay_date.rename(columns={'EggLayDate': 'egg_lay_date'}, inplace=True)

# nesting date: 31 days before egg_lay_date
gb_lay_date['courting_date'] = gb_lay_date['egg_lay_date'] - datetime.timedelta(days=31)

# hatch_date
def hatch_date(row):
    return row['observation_date'] - datetime.timedelta(days=row['ChicksAge'])
# get the observation date (select columns)                    where age is not blank (i.e. they're there)
gb_hatch_date = df_breeding[['nest_id', 'year', 'clutch', 'observation_date', 'ChicksAge']].loc[df_breeding['ChicksAge'].notnull()]
gb_hatch_date['hatch_date'] = gb_hatch_date.apply(hatch_date, axis=1)
# get the min hatch_date 
gb_hatch_date = gb_hatch_date[['nest_id', 'year', 'clutch', 'hatch_date']].groupby(['nest_id', 'year', 'clutch']).min().reset_index()

# fledge_date
# is either the date that the chicks were of age and no longer observed in the nest, or were observed dead
# get the observation date (select columns) where there is a fledge flag
gb_fledge_date = df_breeding[['nest_id', 'year', 'clutch', 'observation_date', 'ChicksAlive', 'ChicksDead', 'ChicksFledge']].fillna(0)
gb_fledge_date['dead_or_fledged'] = gb_fledge_date.apply(lambda row: row['ChicksFledge'] > 0 or (row['ChicksDead'] > 0 and row['ChicksAlive'] == 0), axis=1)
gb_fledge_date = gb_fledge_date.query('dead_or_fledged')
# get the min obs date, which is the earliest fledge recording (per clutch)
gb_fledge_date = gb_fledge_date[['nest_id', 'year', 'clutch', 'observation_date']].groupby(['nest_id', 'year', 'clutch']).min().reset_index()
# rename the obs date 
gb_fledge_date.rename(columns={'observation_date': 'dead_or_fledge_date'}, inplace=True)

# join the key date tables together
print('{0} Merging the phase date tables.'.format(str(time.ctime())), flush=True)
df_phase_dates = pd.merge(left=df_all_nests, right=gb_lay_date, how='left', on=['nest_id'], sort=True)
df_phase_dates = pd.merge(left=df_phase_dates, right=gb_hatch_date, how='left', on=['nest_id', 'year', 'clutch'], sort=True)
df_phase_dates = pd.merge(left=df_phase_dates, right=gb_fledge_date, how='left', on=['nest_id', 'year', 'clutch'], sort=True)

Sun Nov  6 21:04:29 2016 Calculating the breeding phase dates for each nest and year.
Sun Nov  6 21:04:30 2016 Merging the phase date tables.


The `sensor_data` are lacking a `clutch` number, which will create duplicates if we attempt to join on the phase dates. Get the clutch dates and join them into the `sensor_data`.

In [23]:
print('{0} Pivot breeding data to get the clutch dates.'.format(str(time.ctime())), flush=True)
# to avoid making epic dupes, we need to first add the clutch number on to the sensor data table
# get the required cols
df_clutch_pivot = gb_lay_date[['nest_id', 'year', 'clutch', 'egg_lay_date']].copy()
# we have to combine the index because pivot() does not like a multi-index
df_clutch_pivot['nestyear'] = df_clutch_pivot['nest_id'] + '-' + df_clutch_pivot['year'].apply(lambda x: str(x))
# drop the old index fields
df_clutch_pivot = df_clutch_pivot = df_clutch_pivot[['nestyear', 'clutch', 'egg_lay_date']]
# do the pivot to get the (up to three) clutch dates per nest and year
df_clutch_pivot = df_clutch_pivot.pivot(index='nestyear', columns='clutch')['egg_lay_date'].reset_index()
# rename and restore the indexes
df_clutch_pivot.rename(columns={1.0: 'clutch_1', 2.0: 'clutch_2', 3.0: 'clutch_3'}, inplace=True)
df_clutch_pivot['nest_id'] = df_clutch_pivot['nestyear'].apply(lambda x: x.split('-')[0])
df_clutch_pivot['breeding_year'] = df_clutch_pivot['nestyear'].apply(lambda x: int(x.split('-')[1]))
df_clutch_pivot = df_clutch_pivot[['nest_id', 'breeding_year', 'clutch_1', 'clutch_2', 'clutch_3']]

print('{0} Join the clutch dates to the sensor data.'.format(str(time.ctime())), flush=True)
# join on to teh sensor data
df_sensor_clutch = pd.merge(left=df_sensor_data, right=df_clutch_pivot, how='left', on=['nest_id', 'breeding_year'], sort=True)
print('{0} Done. Rows: {1:,}'.format(str(time.ctime()), len(df_sensor_clutch)), flush=True)

print('{0} Assigning a clutch number to each sensor record. Be patient.'.format(str(time.ctime())), flush=True)
# flag each reading with a clutch number
def clutch_number(row):
    if pd.isnull(row['clutch_1']):
        # there are no breeding observations for this nest and year
        return 0
    else:
        # there is at least 1 clutch
        if pd.isnull(row['clutch_2']) or row['datetime'] < row['clutch_2']:
            # there was only a single clutch, or there were >1 but this reading was before the 2nd clutch
            return 1
        elif pd.isnull(row['clutch_3']) or (not pd.isnull(row['clutch_3']) and row['datetime'] < row['clutch_3']):
            # there is a 2nd clutch if we got this far. if there is no 3rd, or the reading is before the 3rd, then this is 2nd
            return 2
        else:
            # there is a 3rd clutch and the sensor reading is after the 3rd
            return 3

df_sensor_clutch['clutch_number'] = df_sensor_clutch.apply(lambda row: clutch_number(row), axis=1)
print('{0} Done.'.format(str(time.ctime())), flush=True)

Sun Nov  6 21:04:30 2016 Pivot breeding data to get the clutch dates.
Sun Nov  6 21:04:30 2016 Join the clutch dates to the sensor data.
Sun Nov  6 21:04:30 2016 Done. Rows: 2,173,738
Sun Nov  6 21:04:30 2016 Assigning a clutch number to each sensor record. Be patient.
Sun Nov  6 21:08:29 2016 Done.


### Get the breeding phase against each sensor reading
1. Join the phase dates on to the sensor data
2. Use the phase date to calculate the breeding_phase for each sensor reading

In [24]:
print('{0} Join the phase dates on to the sensor data.'.format(str(time.ctime())), flush=True)
df_sensor_phase = pd.merge(left=df_sensor_clutch,
                        right=df_phase_dates,
                        how='left',
                        left_on=['nest_id', 'breeding_year', 'clutch_number'], # same key names: don't need to specify R and L
                        right_on=['nest_id', 'year', 'clutch'], # same key names: don't need to specify R and L
                        sort=True # for efficiency do/not sort the df first
#                             suffixes=['_temp', '_humd']
                        )
print('{0} Done. Rows: {1:,}'.format(str(time.ctime()), len(df_sensor_phase)), flush=True)

Sun Nov  6 21:08:29 2016 Join the phase dates on to the sensor data.
Sun Nov  6 21:08:30 2016 Done. Rows: 2,173,738


In [25]:
print('{0} Calculating the breeding_phase for each sensor reading. Be patient.'.format(str(time.ctime())), flush=True)
# for each sensor reading, determine the breeding_phase:
# 'courting' iff date between nesting_date and egg_lay_date
# 'incubating' iff date between egg_lay_date and hatch_date
# 'rearing' iff date between hatch_date adn fledge_date
# 'courting' iff clutch < clutch_count and date between fledge_date and egg_lay_date
# else 'unoccupied' 

def breeding_phase(row):
    if pd.isnull(row['egg_lay_date']) or row['clutch_number'] == 0: 
        # no activity this year
        return 'unoccupied'
    
    elif pd.isnull(row['hatch_date']):
        # laid but never hatched
        if row['datetime'] <= row['egg_lay_date'] + datetime.timedelta(days=35):
            # this egg never hatches, but the current sensor period is incubation
            return 'incubating'
        else:
            # this egg never hatches, and the current sensor period is past the 35 day incubation period
            return 'unoccupied'
    
    elif pd.isnull(row['dead_or_fledge_date']): 
        # hatched but never fledged
        if row['datetime'] <= row['hatch_date'] + datetime.timedelta(days=80):
            # oldest chick at fledge was 77 days, so assume up to 80
            return 'rearing'
        else:
            # the chicks must be missing
            return 'unoccupied' 
    
    elif row['datetime'] < row['courting_date']:
        # no one has moved in yet
        return 'unoccupied'
    
    elif row['clutch_number'] == 1 and row['courting_date'] <= row['datetime'] <= row['egg_lay_date']:
        # for the first clutch, courting is 31 days prior to lay
        return 'courting'
    
    elif row['clutch_number'] > 1 and row['datetime'] <= row['egg_lay_date']:
        # consider it courting again between fledging and second clutch
        return 'courting'
    
    elif row['egg_lay_date'] <= row['datetime'] <= row['hatch_date']:
        return 'incubating'
    
    elif row['hatch_date'] <= row['datetime'] <= row['dead_or_fledge_date']:
        return 'rearing'
    
    elif row['datetime'] > row['dead_or_fledge_date']:
        return 'unoccupied'
    
    else:
        return 'undefined'

df_sensor_phase['breeding_phase'] = df_sensor_phase.apply(lambda row: breeding_phase(row), axis=1)
# tempset['breeding_phase'] = tempset.apply(lambda row: breeding_phase(row), axis=1)
print('{0} Done.'.format(str(time.ctime())), flush=True)

Sun Nov  6 21:08:30 2016 Calculating the breeding_phase for each sensor reading. Be patient.
Sun Nov  6 21:13:43 2016 Done.


### ----------------------------------------------------------------
# Dev and Test
### ----------------------------------------------------------------

In [26]:
# df_sensor_phase.to_csv('df_sensor_phase')
df_sensor_phase.to_pickle('df_sensor_phase.pkl')

In [ ]:
df_sensor_phase = read_pickle('df_sensor_phase.pkl')

In [ ]:
test = df_sensor_phase[['nest_id', 'year', 'temp_c']].groupby(['nest_id', 'year']).describe(include='all')
test.rename(columns={'temp_c':name}).squeeze() for name, group in test)

### Calculate the annual microclimate stats for each nest
This is used to understand the annual nest output absed on it's characteristics in the breeding year

In [ ]:
# get the YEARLY temp and humidity mean, min, max, stddev for each nest and year


# get the PHASE temp and humidity mean, min, max, stddev for each nest, year, clutch and phase

# get the temp and humidity buckets for each nest, year, clutch, phase



### Calculate the microclimate stats for each nest, year and clutch as well as per-phase 
This is used to understand how the microclimate affects the outcome of each clutch and nest selection during courting

In [ ]:
# join the sensor stats (annual and per-phase) onto the nest_and_breeding data



# -------------------------

In [ ]:
# tempset = df_sensor_phase.query('nest_id in ["E10", "E13", "E14", "E4", "T5", "W2", "W6"]')

In [ ]:
# TESTING ONLY

# temp = df_sensor_phase.query('nest_id in ["E10", "E13", "E14", "E4", "T5", "W2", "W6"]')
# temp = tempset.query('nest_id in ["E10", "E13", "E14", "E4", "T5", "W2", "W6"]')
# temp['date'] = temp['datetime'].apply(pd.datetools.normalize_date)
# temp = temp[['nest_id', 'date', 'breeding_year','clutch_number', 
#        'egg_lay_date', 'courting_date', 'hatch_date', 'dead_or_fledge_date',
#        'breeding_phase']]
# temp = temp.drop_duplicates()

# print(len(temp))
# temp.to_csv('sensor_phase_test.csv')
#E13 2014 egg never hatched:        works
#E10 2014 normal single fledge:     works
#W6 2014 chick ded:                 works

Hold off making the dummy columns until we need to do the stats. This keeps the file size down and lets us save the csv with buckets rather than dummies.

In [ ]:
print(str(time.ctime()), 'Creating temp and humidity bucket dummy columns.', end='', flush=True)
df_joined = pd.get_dummies(data=df_joined, columns=['temp_bucket', 'humidity_bucket'])
print(' Done.', flush=True)

In [ ]:
print(str(time.ctime()), 'Aggregating data by nest and year.', flush=True)

def percent_of_time(row):
    return 
# group the data by nest_id and breeding year to get the temp and humidity stats per year
temp_aggregations = {
    'temp_c': {
        'temp_count': 'count',
        'temp_avg': 'mean',
        'temp_min': 'min',
        'temp_max': 'max',
        'temp_std_dev': 'std'        
    },
    'humidity': {
        'humidity_count': 'count',
        'humidity_avg': 'mean',
        'humidity_min': 'min',
        'humidity_max': 'max',
        'humidity_std_dev': 'std'  
    },
    'temp_<0': {'bucket_total': 'sum'},
    'temp_0-5': {'bucket_total': 'sum'},
    'temp_5-10': {'bucket_total': 'sum'},
    'temp_10-15': {'bucket_total': 'sum'},
    'temp_15-20': {'bucket_total': 'sum'},
    'temp_20-25': {'bucket_total': 'sum'},
    'temp_25-30': {'bucket_total': 'sum'},
    'temp_30-35': {'bucket_total': 'sum'},
    'temp_35-40': {'bucket_total': 'sum'},
    'temp_40-45': {'bucket_total': 'sum'},
    'temp_45-50': {'bucket_total': 'sum'},
    'temp_50-55': {'bucket_total': 'sum'},
    'temp_55-60': {'bucket_total': 'sum'},
    'temp_60+': {'bucket_total': 'sum'}    
}
df_joined_gb = df_joined.groupby(['nest_id', 'breeding_year']).agg(temp_aggregations)
print(str(time.ctime()), 'Done.', flush=True)

### to add: 
* return the nest_ids and number and type of missing records

In [ ]:
print(str(time.ctime()), 'Checking for missing data.', flush=True)
# check for missing temp or humidity readinga
def missing_data(row):
    if row['temp_c']['temp_count'] > row['humidity']['humidity_count']:
        return 'missing_humidity_data'
    elif row['temp_c']['temp_count'] < row['humidity']['humidity_count']:
        return 'missing_temp_data'
    else:
        return None
df_joined_gb['missing_data'] = df_joined_gb.apply(missing_data, axis=1)

print(str(time.ctime()), 'Done.', flush=True)

In [ ]:
df_joined.head(10)

In [ ]:
df_joined_gb.head(5)

In [ ]:
# df_joined_gb['temp_25-30']['%time'] = df_joined_gb['temp_25-30']['bucket_total'] / df_joined_gb['temp_c']['temp_count']
df_joined_gb['temp_25-30_total'] = df_joined_gb['temp_25-30']['bucket_total']
df_joined_gb['temp_25-30_hours'] = df_joined_gb['temp_25-30_total'] / 4
df_joined_gb['temp_25-30_%'] = df_joined_gb['temp_25-30_total'] / df_joined_gb['temp_c']['temp_count']



In [ ]:
df_joined_gb.head(5)

In [ ]:
df_joined_gb.head(5)

The below sends the data to the PostGres DB.

Currently considering not using the DB at all. While the data maniopulation within the DB via SQL is far easier, keeping the whole project (data load, manipulate, graph) to a single platform and language is a priority.

In [ ]:
# #sending temperature dataframe to the postgres DB
# print("Transferring temperature dataframe to DB..")
# df_temp.to_sql(con=engine, name='penguins_temperature', if_exists='replace')
# print("Uploaded successfully")

# #sending humidity dataframe to the postgres DB
# print("Transferring humidity dataframe to DB..")
# df_humd.to_sql(con=engine, name='penguins_humidity', if_exists='replace')
# print("Uploaded successfully")

# #sending nests dataframe to the postgres DB
# print("Transferring nests dataframe to DB..")
# nests_raw.to_sql(con=engine, name='penguins_nests', if_exists='replace')
# print("Uploaded successfully")